<a href="https://colab.research.google.com/github/daivik05/signature_personality_prediction/blob/main/sign_test_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import cv2
import numpy as np
from keras.models import load_model
import json


In [2]:
from google.colab import files

# Upload the model file and class indices file
uploaded = files.upload()


Saving class_indices.json to class_indices.json
Saving signature personality prediction3.h5 to signature personality prediction3.h5


In [3]:
print("Files uploaded:")
print(os.listdir())


Files uploaded:
['.config', 'signature personality prediction3.h5', 'class_indices.json', 'sample_data']


In [4]:
# Load the saved model
model = load_model('signature personality prediction3.h5')

# Load the class indices
with open('class_indices.json', 'r') as f:
    class_indices = json.load(f)
class_labels = {v: k for k, v in class_indices.items()}


In [5]:
repo_name = 'signature_personality_prediction'
repo_url = f'https://github.com/daivik05/signature_personality_prediction.git'
!git clone $repo_url

Cloning into 'signature_personality_prediction'...
remote: Enumerating objects: 2307, done.
remote: Counting objects: 100% (521/521), done.
remote: Compressing objects: 100% (514/514), done.
remote: Total 2307 (delta 9), reused 513 (delta 4), pack-reused 1786
Receiving objects: 100% (2307/2307), 330.07 MiB | 40.03 MiB/s, done.
Resolving deltas: 100% (10/10), done.
Updating files: 100% (4307/4307), done.


In [18]:
!pip install flask flask-ngrok
!pip install pyngrok


In [19]:
from pyngrok import ngrok

ngrok.set_auth_token('2j0vyEGckvMBGp1AQ6uhDpHp90D_2wDscsJ7TUU6f3v7RvC4o')


In [ ]:
from flask import Flask, request, jsonify
from pyngrok import ngrok
import cv2
import numpy as np
from keras.models import load_model
import json

app = Flask(__name__)

def preprocess_image(image_path):
    img = cv2.imread(image_path)
    if img is None:
        raise ValueError(f"Image not loaded properly. Please check the file path: {image_path}")
    img = cv2.resize(img, (224, 224))
    img = img.astype('float32') / 255.0
    img = np.expand_dims(img, axis=0)
    return img

def detect_features(signature_image):
    if signature_image is None:
        raise ValueError("Image not loaded properly. Please check the file path.")

    gray = cv2.cvtColor(signature_image, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    edges = cv2.Canny(blurred, 50, 150)

    features = {
        "curved_start": False,
        "curved_sharply_end": False,
        "curved_smoothly_end": False,
        "end_streak_increases": False,
        "end_streak_decreases": False,
        "underline": False
    }

    contours, _ = cv2.findContours(edges.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if len(contours) == 0:
        return features

    largest_contour = max(contours, key=cv2.contourArea)
    contour_length = len(largest_contour)

    start_point = largest_contour[0][0]
    end_point = largest_contour[-1][0]

    if start_point[1] < start_point[0]:
        features["curved_start"] = True

    if end_point[1] < start_point[1]:
        features["end_streak_increases"] = True
    else:
        features["end_streak_decreases"] = True

    underline_threshold = signature_image.shape[0] - 10
    for point in largest_contour:
        if point[0][1] > underline_threshold:
            features["underline"] = True
            break

    delta_x = abs(end_point[0] - start_point[0])
    delta_y = abs(end_point[1] - start_point[1])
    if delta_x > delta_y:
        features["curved_smoothly_end"] = True
    else:
        features["curved_sharply_end"] = True

    return features

def classify_personality(features):
    personality_traits = []
    if features["curved_start"] and features["curved_sharply_end"]:
        personality_traits.append("Formulates a sharp mind")
    if features["curved_start"] and features["curved_smoothly_end"]:
        personality_traits.append("Friendly and diplomatic")
    if features["end_streak_increases"]:
        personality_traits.append("Confident")
    if features["end_streak_decreases"]:
        personality_traits.append("Lacks spirit, realistic thinking, and confidence")
    if features["underline"]:
        personality_traits.append("Unique idea and thinking, needs support to make decisions, and has reliability in the lead")

    if not personality_traits:
        return "Personality traits not detected"

    return ", ".join(personality_traits)

model = load_model('signature personality prediction3.h5')
with open('class_indices.json', 'r') as f:
    class_indices = json.load(f)
class_labels = {v: k for k, v in class_indices.items()}

@app.route('/upload', methods=['POST'])
def upload_image():
    file = request.files['file']
    file.save('uploaded_image.png')
    test_image_path = 'uploaded_image.png'

    test_image = preprocess_image(test_image_path)

    if test_image is None:
        return jsonify({"error": "Image not loaded properly. Please check the file path."})

    prediction = model.predict(test_image)
    predicted_class_index = np.argmax(prediction)
    predicted_class_label = class_labels[predicted_class_index]

    if "forg" not in predicted_class_label:
        signature_image = cv2.imread(test_image_path)
        features = detect_features(signature_image)
        personality = classify_personality(features)
        return jsonify({"result": "genuine", "personality": personality})
    else:
        return jsonify({"result": "forged"})

if __name__ == '__main__':
    from pyngrok import ngrok


    ngrok.set_auth_token("2j0vyEGckvMBGp1AQ6uhDpHp90D_2wDscsJ7TUU6f3v7RvC4o")


    public_url = ngrok.connect(5000)
    print(" * ngrok tunnel URL:", public_url.public_url)

    app.run()


 * ngrok tunnel URL: https://6397-35-193-104-9.ngrok-free.app
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


1/1 [==============================] - 1s 1s/step


INFO:werkzeug:127.0.0.1 - - [10/Jul/2024 15:31:06] "POST /upload HTTP/1.1" 200 -
